In [ ]:
!pip install torch numpy sklearn


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
num_classes = len(emotion_labels)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
class FaceMeshCSVLoader(Dataset):
    def __init__(self, csv_path):
        data = pd.read_csv(csv_path)
        X = data.iloc[:, :-1].values.reshape(-1, 468, 3).astype(np.float32)  # (N, 468, 3)

        # Normalize per sample
        self.X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-8)
        self.y = data.iloc[:, -1].values.astype(np.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])


In [ ]:
train_dataset = FaceMeshCSVLoader("facemesh_train.csv")
test_dataset = FaceMeshCSVLoader("facemesh_test.csv")

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


In [ ]:
class FaceMeshTransformer(nn.Module):
    def __init__(self, input_dim=3, model_dim=128, num_heads=8, num_layers=4, num_classes=7):
        super().__init__()
        self.input_linear = nn.Linear(input_dim, model_dim)

        self.pos_embedding = nn.Parameter(torch.randn(1, 468, model_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.cls_head = nn.Sequential(
            nn.Linear(model_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.input_linear(x)
        x = x + self.pos_embedding
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        return self.cls_head(x)


In [ ]:
model = FaceMeshTransformer().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [ ]:
def train_model(model, train_loader, test_loader, epochs=15):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)

        train_acc = correct / total
        print(f"Epoch {epoch+1} | Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.4f}")

        # Evaluate
        model.eval()
        test_correct, test_total = 0, 0
        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                out = model(X)
                test_correct += (out.argmax(1) == y).sum().item()
                test_total += y.size(0)

        test_acc = test_correct / test_total
        print(f"           | Test Acc: {test_acc:.4f}")

        scheduler.step()


In [ ]:
train_model(model, train_loader, test_loader, epochs=15)


In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        out = model(X)
        preds = out.argmax(1)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=emotion_labels, yticklabels=emotion_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print(classification_report(y_true, y_pred, target_names=emotion_labels))


In [ ]:
torch.save(model.state_dict(), "facemesh_emotion_transformer.pth")
print("Model saved as facemesh_emotion_transformer.pth")
